 ### -REGREÇÃO LINEAR MÚLTIPLA

É mútipla pois nela são tratadas duas ou mais variáveis explicativas

In [25]:
%matplotlib inline
%matplotlib notebook
%reset -f
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from scipy.stats import norm, probplot
import statsmodels.api as sm
from mpl_toolkits.mplot3d import Axes3D

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)

### Ler o arquivo

In [26]:
df = pd.read_csv("tmdb_5000_movies.csv")
df.drop(df[df['budget']==0].index, inplace=True)

### limpeza do dataframe 

In [27]:
df_novo = df.filter(['budget', 'genres', 'popularity', 'production_companies', 'production_countries',
                     'release_date', 'revenue', 'runtime', 'vote_average', 'vote_count'], axis=1)

### Função utilitária para fazer a regressão com constante adicionada

In [28]:
def regress(X,Y):
    X_cp = sm.add_constant(X)
    model = sm.OLS(Y,X_cp)
    results = model.fit()
    return results

### Analíse dos dados

.............


............


............

Criação de colunas com o nome dos generos, sendo 1 para quando o filme é daquela categoria e 0 para quando não é , isso foi feito para tirar a coluna genre

In [29]:
import json

genres = set()
for x in df_novo['genres']:
    for g in json.loads(x):
        genres.add(g["name"])

genres

{'Action',
 'Adventure',
 'Animation',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Family',
 'Fantasy',
 'Foreign',
 'History',
 'Horror',
 'Music',
 'Mystery',
 'Romance',
 'Science Fiction',
 'TV Movie',
 'Thriller',
 'War',
 'Western'}

criacao do novo dataframe incluindo as novas colunas de genero

In [30]:
def extrai_generos(item):
    generos = {'c_' + x: 0 for x in ['Action', 'Adventure', 'Animation', 'Comedy', 'Crime', 
                              'Documentary', 'Drama', 'Family', 'Fantasy', 'Foreign', 
                              'History', 'Horror', 'Music', 'Mystery', 'Romance', 
                              'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western']}
    for g in json.loads(item):
        generos['c_' + g['name']] += 1
    return generos

df_teste = pd.DataFrame(list(df_novo['genres'].apply(extrai_generos).values))
df_junto = df_novo.join(df_teste)

In [31]:
df_junto.columns

Index(['budget', 'genres', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'vote_average', 'vote_count', 'c_Action', 'c_Adventure', 'c_Animation',
       'c_Comedy', 'c_Crime', 'c_Documentary', 'c_Drama', 'c_Family',
       'c_Fantasy', 'c_Foreign', 'c_History', 'c_Horror', 'c_Music',
       'c_Mystery', 'c_Romance', 'c_Science Fiction', 'c_TV Movie',
       'c_Thriller', 'c_War', 'c_Western'],
      dtype='object')

substituição dos '-' na coluna release date

In [32]:
df_junto['release_date'] = df_junto.release_date.str.replace("-", "")


Obtendo as variáveis resposta e preditora:]


-Variável resposta , ou seja , as notas do site IMDB


-Variável preditora , ou seja , todas as colunas que serão usadas para prever as notas dos filmes

In [33]:
df_sem_nan = df_junto.dropna()

In [34]:
Y = df_sem_nan['vote_average']

In [35]:
X = df_sem_nan[['budget', 'popularity', 'runtime', 'vote_count',]]

In [36]:
contadordezerosbudgets = X.budget.value_counts (True)
contadordezerosrevenue = X.revenue.value_counts (True)
print(contadordezerosbudgets )
print(contadordezerosrevenue)

AttributeError: 'DataFrame' object has no attribute 'revenue'

In [37]:
results = regress(X,Y)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           vote_average   R-squared:                       0.355
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     442.6
Date:                Thu, 08 Nov 2018   Prob (F-statistic):          3.27e-304
Time:                        14:26:27   Log-Likelihood:                -3408.9
No. Observations:                3220   AIC:                             6828.
Df Residuals:                    3215   BIC:                             6858.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.4799      0.069     65.310      0.000       4.345       4.614
budget     -6.704e-09   3.39e-10    -19.794      0.000   -7.37e-09   -6.04e-09
popularity     0.0011      0.001      2.065      0.039    5.38e-05       0.002
runtime        0.0157      0.001     25.146      0.000       0.015       0.017
vote_count     0.0003   1.42e-05     19.781      0.000       0.000       0.000
==============================================================================
Omnibus:                      262.337   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              433.958
Skew:                          -0.604   Prob(JB):                     5.85e-95
Kurtosis:                       4.333   Cond. No.                     3.41e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.41e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Separando o dataframe em teste e treinamento

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=RANDOM_SEED)

Treinando, criando , testando e calculando o modelo de regressão linear

In [39]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()       #cria
model.fit(X_train, y_train)        #treina 
y_pred = model.predict(X_test)      #testa 

1 - mean_squared_error(y_test, y_pred) / np.var(y_test)


0.3825356837716132